# Weather Changes Over Time (Consuming APIs)

## Task
Utilize the following two APIs to evaluate the changes in temperature for your birthday and city you were born in:
* [geocode.xyz](geocode.xyz)
* [open-meteo.com](open-meteo.com)


In [1]:
from datetime import date, datetime, timedelta

import requests
import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import pyplot as plt
from matplotlib.ticker import FormatStrFormatter

%matplotlib inline

sns.set_palette('muted')
sns.set_style('white')

## Question 1: Geocoding A Location

> Indented block


Write a function that utilizes the [geocode.xyz](geocode.xyz) API to take in a city and country and return the latitude and longitude of that location

```
def geocode_location(city, country):
    # your code
    return latitude, longitude
```

For example:
```
geocode_location("london", "united kingdom")
```
should return
```
(51.49742, -0.11534)
```

In [2]:
# write your solution here
def geocode_location(city,country):
    url = f"http://geocode.xyz/{city},{country}?json=1"
    response = requests.get(url).json()
    latitude = response['latt']
    longitude = response ['longt']
    return latitude, longitude

In [3]:
geocode_location("Detroit", "United States")

('42.38822', '-83.10357')

## Question 2: Determining Weather Using Date and Location
Write a function that utilizes the [open-meteo.com](https://open-meteo.com/) API to take in a date and location and return the 15-day average high and low for the following measurements from the years 1960 to 2022:
* `precipitation_sum` (mm): Sum of daily precipitation (including rain, showers and snowfall)
* `temperature_2m_max` (°F): Maximum daily air temperature at 2 meters above ground
* `temperature_2m_min` (°F): Minimum daily air temperature at 2 meters above ground
* `apparent_temperature_max` (°F): Maximum daily apparent temperature
* `apparent_temperature_min` (°F): Min daily apparent temperature

The function should return the above measurements in a dictionary object with the keys being `"precipitation_sum"`, `"temperature_2m_max"`, etc. and the values being the 15-day average of the measurements. The 15-day average should be the 7 days preceeding the input date, the input date, and the 7 days following the input date. This will give the results more statistical validity by increasing the number of samples for each measurement.

The `get_historical_weather_measurements()` function will include a few different sub-tasks:

### Create API Request in Postman/Insomnia
Use the following [API Documentation](https://open-meteo.com/en/docs/historical-weather-api#api-documentation) page to create an API request that responds with a JSON object of the measurements listed above. Make sure the response object meets the following:
* Temperatue measurements are in Fahrenheit
* Precipitation measurements are in inches
* Measurements are daily aggregations
* 15 day range of weather
* JSON format

Before writing any code, use Postman/Insomnia to create the API request and take a screenshot of the request/response.

#### HINT: How to insert a screenshot
1. Take a screenshot and save it to a file.
2. Create a markdown cell in your notebook.
3. Insert the image into the markdown cell.
  * Option 1: Drag the image file into the markdown cell. This embeds the image data directly into your Jupyter notebook.
  * Option 2: Move the file into the directory with your notebook file (inside the project folder that you will submit). Add markdown to insert the image: `![alternative text](path-to-image)`

### Create API Request Function
Convert the request above into a function. The function should take in `latitude`, `longitude`, and `date` as its arguments and return the API response, if successful.

```
def get_api_response(latitude, longitude, date):
    # your code

```

Use the helper function below for creating a 15-day date range

In [4]:
# helper function for 15-day date range
def get_date_range(birthday):
    start_date = (birthday - timedelta(days=7)).strftime("%Y-%m-%d")
    end_date = (birthday + timedelta(days=7)).strftime("%Y-%m-%d")
    return {"start_date": start_date, "end_date": end_date}

# Example usage
birthday = datetime.strptime("1998-09-08", "%Y-%m-%d")
print(get_date_range(birthday))

{'start_date': '1998-09-01', 'end_date': '1998-09-15'}


In [5]:
# write your solution here
def get_api_response(latitude, longitude, date):
    date_obj = datetime.strptime(date, "%Y-%m-%d")
    date_range = get_date_range(date_obj)
    base_url = "https://archive-api.open-meteo.com/v1/archive"
    params = {
        "latitude": latitude,
        "longitude": longitude,
        "start_date": date_range["start_date"],
        "end_date": date_range["end_date"],
        "daily": [
            "precipitation_sum",
            "temperature_2m_max",
            "temperature_2m_min",
            "apparent_temperature_max",
            "apparent_temperature_min",
        ],
        "temperature_unit": "fahrenheit",
        "precipitation_unit": "inch",
        "timezone": "auto",
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        return response.json()
    else:
        # Handle errors gracefully
        raise Exception(f"API request failed with status code {response.status_code}: {response.text}")
        
latitude = 42.38822 
longitude = -83.10357 
date = "1998-09-08"

try:
    weather_data = get_api_response(latitude, longitude, date)
    print(weather_data)
except Exception as e:
    print(str(e))

    pass # remove this line once you start adding code to your function

{'latitude': 42.355007, 'longitude': -83.13785, 'generationtime_ms': 0.18596649169921875, 'utc_offset_seconds': -18000, 'timezone': 'America/Detroit', 'timezone_abbreviation': 'EST', 'elevation': 199.0, 'daily_units': {'time': 'iso8601', 'precipitation_sum': 'inch', 'temperature_2m_max': '°F', 'temperature_2m_min': '°F', 'apparent_temperature_max': '°F', 'apparent_temperature_min': '°F'}, 'daily': {'time': ['1998-09-01', '1998-09-02', '1998-09-03', '1998-09-04', '1998-09-05', '1998-09-06', '1998-09-07', '1998-09-08', '1998-09-09', '1998-09-10', '1998-09-11', '1998-09-12', '1998-09-13', '1998-09-14', '1998-09-15'], 'precipitation_sum': [0.004, 0.126, 0.0, 0.0, 0.0, 0.0, 0.362, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.323], 'temperature_2m_max': [76.2, 74.0, 75.3, 76.0, 82.5, 90.5, 77.1, 65.0, 67.4, 73.9, 84.1, 86.1, 85.1, 82.6, 79.1], 'temperature_2m_min': [60.0, 61.5, 56.5, 58.5, 58.2, 68.5, 62.3, 55.0, 54.4, 50.1, 59.3, 62.6, 62.7, 66.7, 70.6], 'apparent_temperature_max': [75.1, 69.4, 72.

### Iterating Over Each Year
def get_all_response_objects(latitude, longitude, date, start_year, end_year):
    """
    Fetch historical weather data for each year in the range from start_year to end_year.
    Args:
        latitude (float): Latitude of the location.
        longitude (float): Longitude of the location.
        date (str): The target date in MM-DD format.
        start_year (int): The starting year for iteration.
        end_year (int): The ending year for iteration.
    Returns:
        list: A list of JSON response objects containing weather data for each year.
    """
    responses = []  # To store the response objects
    for year in range(start_year, end_year + 1):
        # Generate the full date for the year
        full_date = f"{year}-{date}"
        try:
            # Fetch the API response for the specific year
            response = get_api_response(latitude, longitude, full_date)
            responses.append(response)
        except Exception as e:
            print(f"Error fetching data for {year}: {e}")
    
    return responses

# Example usage
latitude = 42.3314  # Latitude for Detroit, MI
longitude = -83.0458  # Longitude for Detroit, MI
date = "07-15"  # MM-DD format
start_year = 1960
end_year = 2022

# Fetch all response objects
all_responses = get_all_response_objects(latitude, longitude, date, start_year, end_year)
print(f"Fetched data for {len(all_responses)} years")




In [6]:
# write your solution here
def get_all_response_objects(latitude, longitude, date):

    start_year = 1960
    end_year = 2022
    responses = []  # To store the response objects
    
    for year in range(start_year, end_year + 1):
        full_date = f"{year}-{date}"
        try:
            response = get_api_response(latitude, longitude, full_date)
            responses.append(response)
        except Exception as e:
            print(f"Error fetching data for {year}: {e}")
    
    return responses


latitude = 42.3314  
longitude = -83.0458 
date = "09-08"  

all_responses = get_all_response_objects(latitude, longitude, date)

print(f"Fetched data for {len(all_responses)} years")


Fetched data for 63 years


### Parse JSON Responses & Aggregate Data
Write a function that takes the list of JSON response objects from the previous function in as a parameter and returns a pandas DataFrame containing the original input date, the measurements, and the average across the 15 days.
```
def parse_json_response(responses):
    # your code
```


In [7]:
import pandas as pd

def parse_json_response(responses):
    """
    Parse the list of JSON response objects and return a DataFrame with averages for each measurement.
    Args:
        responses (list): List of JSON response objects from the API.
    Returns:
        pandas.DataFrame: A DataFrame with the input date and averages for the measurements.
    """
    # Initialize an empty list to hold the parsed data
    data = []

    # Iterate through each response
    for response in responses:
        try:
            # Extract daily data
            daily_data = response.get("daily", {})
            
            # Extract measurements
            dates = daily_data.get("time", [])
            precipitation = daily_data.get("precipitation_sum", [])
            temp_max = daily_data.get("temperature_2m_max", [])
            temp_min = daily_data.get("temperature_2m_min", [])
            apparent_temp_max = daily_data.get("apparent_temperature_max", [])
            apparent_temp_min = daily_data.get("apparent_temperature_min", [])
            
            # Verify data lengths match
            if len(dates) == len(precipitation) == len(temp_max) == len(temp_min) == len(apparent_temp_max) == len(apparent_temp_min):
    
                for i in range(len(dates)):
                    data.append({
                        "date": dates[i],
                        "precipitation_sum": precipitation[i],
                        "temperature_2m_max": temp_max[i],
                        "temperature_2m_min": temp_min[i],
                        "apparent_temperature_max": apparent_temp_max[i],
                        "apparent_temperature_min": apparent_temp_min[i],
                    })
            else:
                print(f"Data length mismatch in response: {response}")
        except Exception as e:
            print(f"Error parsing response: {e}")
    
    df = pd.DataFrame(data)

    result = {
        "precipitation_sum_avg": df["precipitation_sum"].mean(),
        "temperature_2m_max_avg": df["temperature_2m_max"].mean(),
        "temperature_2m_min_avg": df["temperature_2m_min"].mean(),
        "apparent_temperature_max_avg": df["apparent_temperature_max"].mean(),
        "apparent_temperature_min_avg": df["apparent_temperature_min"].mean(),
    }

    avg_df = pd.DataFrame([result])

    return avg_df

    

In [8]:
df = parse_json_response(all_responses)

print(df)


   precipitation_sum_avg  temperature_2m_max_avg  temperature_2m_min_avg  \
0               0.081961               74.991323               61.268889   

   apparent_temperature_max_avg  apparent_temperature_min_avg  
0                     75.033016                     60.054603  


### Combining the Functions
Write a function that combines all the above functions into one, where the user inputs their birthday and location and the function returns the pandas DataFrame containing the summary of measurements.

```
def get_data():
    birthday = get_user_birthday()
    location = get_user_city_of_birth()
    # your code here
```

The helper functions below can be used for birthday and location input information:

In [9]:
def get_user_birthday():
    while True:
        birthday = input("Enter your birthday (MM-DD): ")
        try:
            # Validate the format
            datetime.strptime(birthday, "%m-%d")
            return birthday
        except ValueError:
            print("Invalid format. Please enter in MM-DD format (e.g., 07-15).")

def get_user_city_of_birth():
    city = input("Enter your city of birth: ").strip()
    country = input("Enter your country of birth: ").strip()
    return city, country


In [ ]:
def get_data():
    birthday = get_user_birthday()
    city, country = get_user_city_of_birth()
    
    try:
        latitude, longitude = geocode_location(city, country)
    except Exception as e:
        print(f"Error geocoding location: {e}")
        return None
    
    try:
        all_responses = get_all_response_objects(latitude, longitude, birthday)
    except Exception as e:
        print(f"Error fetching data: {e}")
        return None
    
    try:
        summary_df = parse_json_response(all_responses)
    except Exception as e:
        print(f"Error parsing data: {e}")
        return None

    return summary_df


Run your function and set it equal to `df`:

In [ ]:
summary_df = get_data()

if summary_df is not None:
    print("Summary of Weather Measurements:")
    print(summary_df)

## Question 3: Visualizing The Data
Plot the DataFrame using `matplotlib`/`seaborn` and write a short summary of your results.

In [ ]:
def visualize_data(df):
    sns.set(style = "whitegrid")

    melted_df = df.melt(var_name = "Measurement", value_name = "Average Value")

    plt.figure(figsize=(10, 6))
    sns.barplot(
        data = melted_df,
        x = "Measurement",
        y = "Average Value",
        palette  ="coolwarm",
    )

    plt.xticks(rotation = 45, ha = "right")

    plt.title("15-Day Average Weather Measurements (1960-2022)", fontsize = 16)
    plt.xlabel("Measurement", fontsize = 12)
    plt.ylabel("Average Value", fontsize = 12)

    plt.tight_layout()
    plt.show()

if df is not None:
    visualize_data(df)
else:
    print("DataFrame is empty. Unable to visualize data.")


### Results Summary: